In [1]:
import os
import requests
from utils.sl_models import engine as sl_engine
from utils.bodhi_models import engine as bodhi_engine
from utils.bodhi_models import get_session, SpotsModel, WaveForecastModel
from sqlalchemy import select, and_, not_, text
from pydantic import BaseModel, ConfigDict
import pandas as pd

### Test Pydantic Models

In [2]:
with get_session(bodhi_engine) as db:
    stmt = """select * from wave_forecast limit 1"""
    results = db.execute(stmt).fetchall()

/tmp/ipykernel_84535/62947810.py:3: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  results = db.execute(stmt).fetchall()


In [3]:
results

[(20132278, '0101000020E610000000000000000023400000000000F05340', 79.75, 9.5, datetime.datetime(2024, 6, 10, 0, 0, tzinfo=datetime.timezone.utc), datetime.timedelta(0), datetime.datetime(2024, 6, 10, 0, 0, tzinfo=datetime.timezone.utc), 1.090000033378601, 5.170000076293945, 46.459999084472656, 1.0399999618530273, 5.139999866485596, 44.43000030517578, 8.420000076293945, 63.06999969482422, 0.33000001311302185, 6.800000190734863, datetime.datetime(2024, 6, 10, 16, 22, 15, 153926, tzinfo=datetime.timezone.utc))]

In [4]:
[WaveForecastModel.model_validate(entry._asdict()) for entry in results]

[WaveForecastModel(id=20132278, location='0101000020E610000000000000000023400000000000F05340', latitude=79.75, longitude=9.5, time=datetime.datetime(2024, 6, 10, 0, 0, tzinfo=datetime.timezone.utc), step=datetime.timedelta(0), valid_time=datetime.datetime(2024, 6, 10, 0, 0, tzinfo=datetime.timezone.utc), swh=1.090000033378601, perpw=5.170000076293945, dirpw=46.459999084472656, shww=1.0399999618530273, mpww=5.139999866485596, wvdir=44.43000030517578, ws=8.420000076293945, wdir=63.06999969482422, swell=0.33000001311302185, swper=6.800000190734863, entry_updated=datetime.datetime(2024, 6, 10, 16, 22, 15, 153926, tzinfo=datetime.timezone.utc))]

In [5]:
class SpotSpatialIdx(BaseModel):
    spot_id: str
    spot_lat: float
    spot_lon: float

    model_config = ConfigDict(from_attributes=True)

In [6]:
with get_session(sl_engine) as db:
    stmt = "select spot_id, spot_lat, spot_lon from sl_spots"
    results = db.execute(stmt).fetchall()


In [7]:
spatial_idxs = [SpotSpatialIdx.model_validate(entry) for entry in results]

In [8]:
with get_session(sl_engine) as db:
    stmt = text("""select distinct on ("associated_spotId") "associated_spotId", "associated_offshoreLocation_lat", "associated_offshoreLocation_lon" from sl_ratings""")
    results = db.execute(stmt).fetchall()

In [9]:
class SlOffshoreIdx(BaseModel):
    associated_spotId: str
    associated_offshoreLocation_lat: float
    associated_offshoreLocation_lon: float

    model_config = ConfigDict(from_attributes=True)

In [10]:
data = [SlOffshoreIdx.model_validate(entry) for entry in results]

In [11]:
data_dicts = [SlOffshoreIdx.model_dump(entry) for entry in data]

In [12]:
df = pd.DataFrame(data_dicts)

In [13]:
df.head()

,associated_spotId,associated_offshoreLocation_lat,associated_offshoreLocation_lon
0,5842041f4e65fad6a77087f9,37.50,-122.75
1,5842041f4e65fad6a7708804,45.25,-124.25
2,5842041f4e65fad6a7708805,36.75,-122.25
3,5842041f4e65fad6a7708806,36.75,-122.25
4,5842041f4e65fad6a7708807,36.90,-122.10


Create a mask to only keep lat an lon where they are in the intervals .0, .25, .5, .75

In [14]:
df['lat_mod'] = df['associated_offshoreLocation_lat'] % 4
df['lon_mod'] = df['associated_offshoreLocation_lon'] % 4

In [15]:
mask = (df['lat_mod'].apply(lambda x: round(x, 2) == x) & df['lon_mod'].apply(lambda x: round(x, 2) == x))

In [16]:
df = df[mask]

In [17]:
df = df.drop(columns=['lat_mod', 'lon_mod'])

In [18]:
len(df)

582

In [19]:
lat_lon_list = list(zip(df['associated_offshoreLocation_lat'].values, df['associated_offshoreLocation_lon'].values))

In [20]:
lat_lon_str = ', '.join(map(str, lat_lon_list))

In [21]:
with get_session(bodhi_engine) as db:
    stmt = text("""CREATE INDEX if not exists idx_wave_forecast_lat_lon ON wave_forecast (latitude, longitude)""")
    results = db.execute(stmt)
    db.commit()

In [22]:
with get_session(bodhi_engine) as db:
    stmt = text("""reindex index idx_wave_forecast_lat_lon""")
    results = db.execute(stmt)
    db.commit()

In [23]:
with get_session(bodhi_engine) as db:
    stmt = text(f"""select  from wave_forecast where time = CURRENT_DATE AND (latitude, longitude) in ({lat_lon_str}) limit 5""")
    results = db.execute(stmt).fetchall()

In [24]:
results

[(32601315, '0101000020E610000000000000008061C00000000000C04D40', 59.5, -140.0, datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), datetime.timedelta(0), datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), 1.9600000381469727, 10.489999771118164, 218.67999267578125, None, None, None, 4.110000133514404, 269.8800048828125, 1.8700000047683716, 10.489999771118164, datetime.datetime(2024, 6, 12, 10, 56, 37, 64720, tzinfo=datetime.timezone.utc)),
 (32605384, '0101000020E610000000000000000063C00000000000E04C40', 57.75, -152.0, datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), datetime.timedelta(0), datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), 1.3300000429153442, 8.25, 162.5, 0.009999999776482582, 0.20000000298023224, 2.1500000953674316, 3.809999942779541, 84.05000305175781, 1.2000000476837158, 8.210000038146973, datetime.datetime(2024, 6, 12, 10, 56, 37, 64720, tzinfo=datetime.timezone.utc)),
 (32605984, '0101000020E6100